# DMS_QL

In [1]:
import keras

Using TensorFlow backend.


In [2]:
from __future__ import print_function, division

In [3]:
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
from annoy import AnnoyIndex
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [4]:
from keras.layers import Conv1D, Input, Add, Activation, Dropout, Embedding, MaxPooling1D, \
    GlobalMaxPool1D, Flatten, Dense, Concatenate, BatchNormalization
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.initializers import TruncatedNormal
from keras.layers.advanced_activations import LeakyReLU, ELU
from keras import optimizers

In [5]:
from methods.baseline import Baseline
from methods.experiments import Experiment
from methods.evaluation import Evaluation
from methods.retrieval import Retrieval

## Configurações Globais

In [6]:
MAX_SEQUENCE_LENGTH_T = 20 # 100
MAX_SEQUENCE_LENGTH_D = 20 # 500
EMBEDDING_DIM = 300
MAX_NB_WORDS = 20000
'''
    Configuration
'''
epochs = 1000
best_loss = 1
best_epoch = 0
verbose = 0
loss = 1

### Parse bugs preproprecessed

In [7]:
# Domain to use
DOMAIN = 'eclipse'
METHOD = 'DMS_QL_{}'.format(epochs)
# Dataset paths
DIR = 'data/processed/{}'.format(DOMAIN)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Glove embeddings
GLOVE_DIR='data/embed'
# Save model
SAVE_PATH = '{}_feature@number_of_epochs@epochs_64batch({})'.format(METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_feature_@number_of_epochs@epochs_64batch({})'.format(METHOD, DOMAIN)

# Extract CORPUs
EXTRACT_CORPUS = False

In [8]:
import os

pretrained_path = 'uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
model_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [9]:
from keras_bert import load_vocabulary

token_dict = load_vocabulary(vocab_path)

In [10]:
baseline = Baseline(DOMAIN, DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D,
                   token_dict['[CLS]'], token_dict['[SEP]'])
evaluation = Evaluation(verbose=0)
retrieval = Retrieval()
experiment = Experiment(baseline, evaluation)

In [11]:
experiment.set_retrieval(retrieval, baseline, DOMAIN)

Creating the buckets...


#### Loading bug ids in memory

In [12]:
experiment.load_ids()
len(baseline.bug_ids)

Reading bug ids


361006

#### Dicionário de títulos e descrições

In [13]:
%%time

experiment.load_bugs()
len(baseline.sentence_dict)


CPU times: user 31 s, sys: 3.6 s, total: 34.6 s
Wall time: 34.6 s


#### Hashing bugs by buckets

In [14]:
issues_by_buckets = experiment.get_buckets_for_bugs()

#### Prepare the train and test

In [15]:
# path_train='train_chronological', path_test='test_chronological'
experiment.prepare_dataset(issues_by_buckets, path_train='train_chronological', path_test='test_chronological')
# Read and create the test queries duplicates
retrieval.create_queries()

Reading train data
Reading bug ids


In [16]:
baseline.train_data[:10]

[[1, 183],
 [15392, 2],
 [15392, 9779],
 [15392, 94],
 [2, 9779],
 [2, 94],
 [9779, 94],
 [42962, 7],
 [10, 121067],
 [40, 20]]

#### Recovery bug ids from train

In [17]:
bug_train_ids = experiment.get_train_ids(baseline.train_data)

### Export the corpus train

In [18]:
if EXTRACT_CORPUS:
    corpus = []
    export_file = open(os.path.join(DIR, 'corpus_train.txt'), 'w')
    for bug_id in tqdm(baseline.bug_set):
        bug = baseline.bug_set[bug_id]
        title = bug['title']
        desc = bug['description']
        export_file.write("{}\n{}\n".format(title, desc))
    export_file.close()

# Generating tiple of batches

In [19]:
idx = np.random.choice(baseline.bug_ids, 1)[0]
baseline.bug_set[idx]

{'bug_severity': '2\n',
 'bug_status': '0\n',
 'component': '795\n',
 'creation_ts': '2006-07-05 14:03:00 -0400',
 'delta_ts': '2007-03-28 03:39:03 -0400',
 'description': '[CLS] when installing bi ##rt 2 . 1 using the update manager , the user receives a number of warnings ( one for each " feature " , i think ) indicating the feature is not signed . this is quiet a disco ##nce ##rting warning ( see attached ) and may put some user off installing bi ##rt . to see this problem , simply follow the installation instructions from the call ##isto download site . in summary : 1 . install eclipse 3 . 2 id ##e 2 . in update manager , select the bi ##rt features ( and depend ##encies ) 3 . press the " update " button for future releases , we need to make sure that the bi ##rt features are signed in order to avoid this message . ( note that this should also probably be part of the release check ##list to ensure that it is completed for the final builds ) . [SEP]',
 'description_bert': '[CLS] whe

### Generating the batch test

In [20]:
"Train ", len(baseline.dup_sets_train)

('Train ', 34882)

In [21]:
%%time

batch_size = 64
batch_size_test = 128

# we want a constant validation group to have a frame of reference for model performance
batch_triplets_valid, valid_input_sample, valid_input_pos, valid_input_neg, valid_sim = experiment.batch_iterator(None, 
                                                                                                      baseline.train_data, 
                                                                                                      baseline.dup_sets_train,
                                                                                                      bug_train_ids,
                                                                                                      batch_size_test, 1,
                                                                                                      issues_by_buckets)
test_gen = ([valid_input_sample['title'], valid_input_pos['title'], valid_input_neg['title'], 
             valid_input_sample['description'], valid_input_pos['description'], valid_input_neg['description'],
            valid_input_sample['info'], valid_input_pos['info'], valid_input_neg['info']], valid_sim)

# Categorical columns
number_of_columns_info = valid_input_sample['info'].shape[1]
# Max sequence title
MAX_SEQUENCE_LENGTH_T = valid_input_sample['title'].shape[1]
MAX_SEQUENCE_LENGTH_D = valid_input_sample['description'].shape[1]

CPU times: user 66.8 ms, sys: 3.97 ms, total: 70.8 ms
Wall time: 69.9 ms


In [22]:
valid_input_sample['title'].shape, valid_input_sample['description'].shape, valid_input_sample['info'].shape, valid_sim.shape

((128, 20), (128, 20), (128, 1682), (128,))

### Validar entrada

In [23]:
# %%time 

#baseline.display_batch(baseline.train_data, baseline.dup_sets_train, bug_train_ids, 5)

In [24]:
"Test ", len(baseline.test_data)

('Test ', 4641)

## Pre-trained embeddings

Loading pretrained word vectors

### Glove

In [25]:
vocab = baseline.load_vocabulary(os.path.join(DIR, 'vocab_embed.pkl'))
#print(np.random.choice(vocab, 10))
# for token in vocab:
#     print(token)

vocabulary loaded


In [26]:
"Total vocabulary: {}".format(len(vocab))

'Total vocabulary: 21175'

In [27]:
def generating_embed(baseline, GLOVE_DIR, EMBEDDING_DIM):
    embeddings_index = {}
    embed_path = os.path.join(GLOVE_DIR, 'glove.42B.300d.txt')
    f = open(embed_path, 'rb')
    #num_lines = sum(1 for line in open(embed_path, 'rb'))

    vocab = baseline.load_vocabulary(os.path.join(baseline.DIR, 'vocab_embed.pkl'))
    vocab_size = len(vocab) 

    # Initialize uniform the vector considering the Tanh activation
    embedding_matrix = np.random.uniform(-1.0, 1.0, (vocab_size, EMBEDDING_DIM))
    embedding_matrix[0, :] = np.zeros(EMBEDDING_DIM)

    loop = tqdm(f)
    loop.set_description("Loading Glove")
    for line in loop:
        tokens = line.split()
        word = tokens[0]
        embeddings_index[word] = np.asarray(tokens[1:], dtype='float32')
        loop.update(1)
    f.close()
    loop.close()

    print('Total %s word vectors in Glove 42B 300d.' % len(embeddings_index))

    loop = tqdm(total=vocab_size)
    loop.set_description('Loading embedding from dataset pretrained')
    i = 0
    for word, embed in vocab.items():
        if word in embeddings_index:
            embedding_matrix[i] = embeddings_index[word]
        else:
            embedding_matrix[i] = np.asarray(embed, dtype='float32')
        loop.update(1)
        i+=1
    loop.close()
    baseline.embedding_matrix = embedding_matrix

In [28]:
%%time

generating_embed(baseline, GLOVE_DIR=GLOVE_DIR, EMBEDDING_DIM=EMBEDDING_DIM) # MAX_NB_WORDS=MAX_NB_WORDS

vocabulary loaded



Total 1917494 word vectors in Glove 42B 300d.



CPU times: user 1min 21s, sys: 3.23 s, total: 1min 25s
Wall time: 1min 22s


## Experiment

## Towards Accurate Duplicate Bug Retrieval Using Deep Learning Techniques

https://github.com/tqtg/DuplicateBugFinder

### Embedding layer

In [29]:
from keras.constraints import MaxNorm
from keras.initializers import TruncatedNormal, RandomUniform

# Is missing the padding_idx used in pytorch
# https://pytorch.org/docs/stable/_modules/torch/nn/modules/sparse.html
# https://stackoverflow.com/questions/54824768/rnn-model-gru-of-word2vec-to-regression-not-learning
def embedding_layer(embeddings, num_words, embedding_dim, max_sequence_length, trainable):
    embedding_layer = Embedding(num_words,
                                  embedding_dim,
                                  name='embedding_layer',
                                  weights=[embeddings],
                                  embeddings_constraint=MaxNorm(max_value=1, axis=0),
                                  #input_length=max_sequence_length,
                                  input_length=None,
                                  trainable=trainable)
    return embedding_layer

### CNN with filter 3,4,5

In [30]:
import keras
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D

def cnn_model(embedding_layer, max_sequence_length):

    sequence_input = Input(shape=(max_sequence_length,), name='Feature_BugInput')
    #sequence_input = Input(shape=(None,), name='Feature_BugInput')
    embedded_sequences = embedding_layer(sequence_input)

    # Yoon Kim model (https://arxiv.org/abs/1408.5882)
    # best combination filter (3, 4, 5) e 128 e 256
    convs = []
    filter_sizes = [3, 4, 5]
    n_filters = 64

    for index, filter_size in enumerate(filter_sizes):
        l_conv = Conv1D(filters=n_filters, kernel_size=filter_size)(embedded_sequences)
        l_pool = MaxPooling1D(pool_size=filter_size)(l_conv) # index+1
        convs.append(l_pool)

    l_merge = Concatenate(axis=1)(convs)

    # add a 1D convnet with global maxpooling, instead of Yoon Kim model
    #conv = Conv1D(filters=n_filters * 3, kernel_size=3)(l_merge)
    layer = GlobalAveragePooling1D()(l_merge)
    #layer = Flatten()(l_merge)
    layer = Dense(300, activation='tanh')(layer)
    #layer = LeakyReLU()(layer)

    cnn_feature_model = Model(inputs=[sequence_input], outputs=[layer], name = 'FeatureCNNGenerationModel') # inputs=visible

    return cnn_feature_model

### Bi-LSTM

In [31]:
from keras.layers import Dense, Input, LSTM, GRU, Dropout, Bidirectional, GlobalAveragePooling1D, TimeDistributed

def lstm_model(embedding_layer, max_sequence_length):
    number_lstm_units = 75
    rate_drop_lstm = 0
    recurrent_dropout = 0

    sequence_input = Input(shape=(max_sequence_length, ), name='Feature_BugInput')
    #sequence_input = Input(shape=(None, ), name='Feature_BugInput')
    embedded_sequences = embedding_layer(sequence_input)

    left_layer = LSTM(number_lstm_units, return_sequences=True)(embedded_sequences)
    right_layer = LSTM(number_lstm_units, return_sequences=True, go_backwards=True)(left_layer)
    
    lstm_layer = Concatenate()([left_layer, right_layer])
    
    #lstm_layer = TimeDistributed(Dense(50))(lstm_layer)
    #layer = Flatten()(lstm_layer)
    layer = GlobalAveragePooling1D()(lstm_layer)
    layer = Dense(300, activation='tanh')(layer)

    lstm_feature_model = Model(inputs=[sequence_input], outputs=[layer], name = 'FeatureLstmGenerationModel') # inputs=visible

    return lstm_feature_model

### MLP

In [32]:
def mlp_model(input_size):
    info_input = Input(shape=(input_size, ), name='Feature_BugInput')
    input_size = 300
    
    for units in [64, 32]:
        layer = Dense(units, activation='tanh', kernel_initializer='random_uniform')(info_input)
    
    layer = Dense(input_size, activation='tanh')(info_input)
    
    mlp_feature_model = Model(inputs=[info_input], outputs=[layer], name = 'FeatureMlpGenerationModel')
    
    return mlp_feature_model

### Siamese model

In [33]:
from keras import backend as K

'''
    Some loss ideas
    hinge loss Kullback-Leibler
    https://stackoverflow.com/questions/53581298/custom-combined-hinge-kb-divergence-loss-function-in-siamese-net-fails-to-genera
'''

def normalize(x, axis):
    norm = K.sqrt(K.sum(K.square(x), axis=axis, keepdims=False))
    return x, K.maximum(norm, K.epsilon())
    
# https://github.com/keras-team/keras/issues/3031
# https://github.com/keras-team/keras/issues/8335
def cosine_distance(inputs):
    x, y = inputs
    x, x_norm = normalize(x, axis=-1)
    y, y_norm = normalize(y, axis=-1)
    distance = K.sum( x * y, axis=-1) / (x_norm * y_norm)
    distance = (distance + K.constant(1)) / K.constant(2)
    # Distance goes from 0 to 2 in theory, but from 0 to 1 if x and y are both
    # positive (which is the case after ReLU activation).
    #return K.mean(distance, axis=-1, keepdims=False)
    return K.mean(distance, axis=-1, keepdims=True)
    
def euclidean_distance(vects):
    x, y = vects
    distance = K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), K.epsilon()))
    # Normalize https://stats.stackexchange.com/questions/53068/euclidean-distance-score-and-similarity
    distance = K.constant(1) / (K.constant(1) + distance)
    return K.mean(distance, keepdims=False)
    #return K.mean(distance, axis=-1, keepdims=False)

def triplet_loss(vects):
    margin = K.constant(1.0)
    pos = vects[0]
    neg = vects[1]
    return K.maximum(0.0, margin + pos - neg)

# https://www.kaggle.com/c/quora-question-pairs/discussion/33631
# https://www.researchgate.net/figure/Illustration-of-triplet-loss-contrastive-loss-for-negative-samples-and-binomial_fig2_322060548
def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    pos = y_pred[0]
    neg = y_pred[1]
    margin = 1
    return K.mean(pos * K.square(neg) +
                  (1 - pos) * K.square(K.maximum(margin - neg, 0)))

def custom_loss(y_true, y_pred):
    return y_pred[0]
def anchor_pos_distance(y_true, y_pred):
    return y_pred[1]
def anchor_neg_distance(y_true, y_pred):
    return y_pred[2]
def anchor_centroid_pos_distance(y_true, y_pred):
    return y_pred[3]
def anchor_centroid_neg_distance(y_true, y_pred):
    return y_pred[4]
def pos_centroid_pos_distance(y_true, y_pred):
    return y_pred[5]
def pos_centroid_neg_distance(y_true, y_pred):
    return y_pred[6]
def neg_centroid_pos_distance(y_true, y_pred): 
    return y_pred[7]
def neg_centroid_neg_distance(y_true, y_pred):
    return y_pred[8]
def triplet_bug(y_true, y_pred):
    return y_pred[9]
def triplet_anchor(y_true, y_pred):
    return y_pred[10]
def triplet_pos(y_true, y_pred):
    return y_pred[11]
def triplet_neg(y_true, y_pred):
    return y_pred[12]
def stack_tensors(vects):
    return K.stack(vects, axis=-1)

In [34]:
from keras.layers import concatenate, Add, Lambda, merge, Average, Maximum
from keras.optimizers import Adam, Nadam

def siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, sequence_length_info, 
                  sequence_length_t, sequence_length_d, name):
  
    bug_t = Input(shape = (sequence_length_t, ), name = 'title_{}'.format(name))
    bug_d = Input(shape = (sequence_length_d, ), name = 'desc_{}'.format(name))
    bug_i = Input(shape = (sequence_length_info, ), name = 'info_{}'.format(name))
    
    bug_t_feat = title_feature_model(bug_t)
    bug_d_feat = desc_feature_model(bug_d)
    bug_i_feat = categorical_feature_model(bug_i)
    
    #bug_feature_output = Add(name = 'merge_features_{}'.format(name))([bug_i_feat, bug_t_feat, bug_d_feat])
    bug_feature_output = concatenate([bug_i_feat, bug_t_feat, bug_d_feat], name = 'merge_features_{}'.format(name))
    
    #     bug_feature_output = Activation('tanh')(bug_feature_output)
    
    # Bug representation layer
    # bug_feature_output = Dense(300, activation='tanh')(bug_feature_output)
    
    bug_feature_model = Model(inputs=[bug_t, bug_d, bug_i], outputs=[bug_feature_output], name = 'merge_features_{}'.format(name))
    
    return bug_feature_model

In [35]:
def siamese_model_centroid(sequence_length, name):
    
    # Bug Centroid Feature
    bug_centroid = Input(shape = (sequence_length, ), name = 'bug_centroid_feature_{}'.format(name))
    
    bug_feature_model = Model(inputs=[bug_centroid], outputs=[bug_centroid], name = 'merge_features_{}'.format(name))
    
    return bug_feature_model

In [36]:
from keras.layers import Average, Reshape, Add
from keras_radam import RAdam
from keras_bert import AdamWarmup, calc_train_steps

def max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, 
                             master_anchor, master_negative, master_positive, 
                         NUMBER_OF_INSTANCES, BATCH_SIZE, EPOCHS, decay_lr=1):
    
    inputs = np.concatenate([encoded_anchor.input, encoded_positive.input, encoded_negative.input], -1).tolist()
    
    inputs.append(master_anchor.input)
    inputs.append(master_positive.input)
    inputs.append(master_negative.input)
    
    encoded_anchor = encoded_anchor.output
    encoded_positive = encoded_positive.output
    encoded_negative = encoded_negative.output
    master_anchor = master_anchor.output
    master_negative = master_negative.output
    master_positive = master_positive.output
    
    # Distance bugs
    positive_d = Lambda(cosine_distance, name='pos_cosine_distance')([encoded_anchor, encoded_positive])
    negative_d = Lambda(cosine_distance, name='neg_cosine_distance')([encoded_anchor, encoded_negative])
    
    # Distance masters anchor
    master_anchor_positive_d = Lambda(cosine_distance, name='pos_master_cosine_distance')([encoded_anchor, master_positive])
    master_anchor_negative_d = Lambda(cosine_distance, name='neg_master_cosine_distance')([encoded_anchor, master_negative])
    
    # Distance master positive
    master_pos_positive_d = Lambda(cosine_distance, name='pos_master_pos_cosine_distance')([encoded_positive, master_positive])
    master_pos_negative_d = Lambda(cosine_distance, name='neg_master_pos_cosine_distance')([encoded_positive, master_negative])
    
    # Distance master negative
    master_neg_positive_d = Lambda(cosine_distance, name='pos_master_neg_cosine_distance')([encoded_negative, master_negative])
    master_neg_negative_d = Lambda(cosine_distance, name='neg_master_neg_cosine_distance')([encoded_negative, master_positive])
    

    # Loss function only works with a single output
    loss_TL = Lambda(triplet_loss, name='triplet_pos_neg')([positive_d, negative_d])
    loss_TL_A = Lambda(triplet_loss, name='triplet_anchor_centroid')([master_anchor_positive_d, master_anchor_negative_d])
    loss_TL_P = Lambda(triplet_loss, name='triplet_pos_centroid')([master_pos_positive_d, master_pos_negative_d])
    loss_TL_N = Lambda(triplet_loss, name='triplet_neg_centroid')([master_neg_positive_d, master_neg_negative_d])
    
    output_median = Average()([loss_TL, loss_TL_A, loss_TL_P, loss_TL_N])
    
    output = concatenate([output_median, positive_d, negative_d, 
                          master_anchor_positive_d, master_anchor_negative_d,
                         master_pos_positive_d, master_pos_negative_d,
                         master_neg_positive_d, master_neg_negative_d,
                         loss_TL, loss_TL_A, loss_TL_P, loss_TL_N])

    similarity_model = Model(inputs = inputs, outputs = [output], name = 'Similarity_Model')

    # setup the optimization process 
    similarity_model.compile(optimizer='adam', loss=custom_loss, 
                                 metrics=[anchor_pos_distance, anchor_neg_distance, 
                                          anchor_centroid_pos_distance, anchor_centroid_neg_distance,
                                         pos_centroid_pos_distance, pos_centroid_neg_distance,
                                         neg_centroid_pos_distance, neg_centroid_neg_distance,
                                         triplet_bug, triplet_anchor, triplet_pos, triplet_neg])

    return similarity_model

In [37]:
%%time
import keras

# Inspired on https://'pastebin.com/TaGFdcBA
# TODO: https://stackoverflow.com/questions/49941903/keras-compute-cosine-distance-between-two-flattened-outputs
keras.backend.clear_session()

# Clear GPU memory
# from numba import cuda
# cuda.select_device(0)
# cuda.close()

# Embeddings
desc_embedding_layer = embedding_layer(embeddings=baseline.embedding_matrix, 
                              num_words=len(vocab), 
                              embedding_dim=EMBEDDING_DIM, 
                              max_sequence_length=MAX_SEQUENCE_LENGTH_D, 
                              trainable=False)
title_embedding_layer = embedding_layer(embeddings=baseline.embedding_matrix, 
                              num_words=len(vocab), 
                              embedding_dim=EMBEDDING_DIM, 
                              max_sequence_length=MAX_SEQUENCE_LENGTH_T, 
                              trainable=False)

# Feature models
'''
    cnn_model
    lstm_model
    mlp_model
'''
desc_feature_model = cnn_model(desc_embedding_layer, MAX_SEQUENCE_LENGTH_D)
title_feature_model = lstm_model(title_embedding_layer, MAX_SEQUENCE_LENGTH_T)
categorical_feature_model = mlp_model(number_of_columns_info)

# Similarity model
encoded_anchor = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'in')
encoded_positive = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'pos')

encoded_negative = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'neg')

# Centroid 
embed_size = K.int_shape(title_feature_model.get_output_at(0))[1] + K.int_shape(desc_feature_model.get_output_at(0))[1] + K.int_shape(categorical_feature_model.get_output_at(0))[1] 

master_anchor = siamese_model_centroid(embed_size, 'master_anchor')
master_pos = siamese_model_centroid(embed_size, 'master_pos')
master_negative = siamese_model_centroid(embed_size, 'master_neg')

NUMBER_OF_INSTANCES = len(baseline.dup_sets_train)
BATCH_SIZE = batch_size
EPOCHS = epochs

similarity_model = max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, 
                                            master_anchor, master_negative, master_pos,
                                            NUMBER_OF_INSTANCES, BATCH_SIZE, EPOCHS, decay_lr=1)

# cnn_feature_model.summary()
# lstm_feature_model.summary()
similarity_model.summary()

'''
    Experiment
'''
for epoch in range(epochs):
    batch_triplet_train, \
        train_input_sample, train_input_pos, train_input_neg, train_master_input, train_master_neg, \
            train_sim = experiment.batch_iterator_bert(encoded_anchor, baseline.train_data, baseline.dup_sets_train, \
                                                       bug_train_ids, 
                                                       batch_size, 1, 
                                                       issues_by_buckets, 
                                                       TRIPLET_HARD=True, USE_CENTROID=True, method='keras')
    
    train_batch = [train_input_sample['title']['token'], train_input_sample['description']['token'], train_input_sample['info'],
                   train_input_pos['title']['token'], train_input_pos['description']['token'], train_input_pos['info'], 
                   train_input_neg['title']['token'], train_input_neg['description']['token'], train_input_neg['info'],
                   train_master_input['centroid_embed'],
                   train_master_input['centroid_embed'],
                   train_master_neg['centroid_embed']]
    
#     if epoch == 10:
#         similarity_model = max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, decay_lr=0.1)
    
    h = similarity_model.train_on_batch(x=train_batch, y=train_sim)
    
    if (epoch+1 == epochs): #(epoch > 1 and epoch % 10 == 0) or (epoch+1 == epochs):
        recall, _, debug = experiment.evaluate_validation_test(retrieval, verbose, encoded_anchor, issues_by_buckets, bug_train_ids)
        print(("Epoch: {}\n" + 
              "Loss: {:.2f}, pos_cosine: {:.2f}, neg_cosine: {:.2f}, TL: {:.2f}\n" +
              "anchor_pos_centroid_cosine: {:.2f}, anchor_neg_centroid_cosine: {:.2f}, TL_A: {:.2f}\n" +
              "pos_pos_centroid_cosine: {:.2f}, pos_neg_centroid_cosine: {:.2f}, TL_A: {:.2f}\n" +
              "neg_pos_centroid_cosine: {:.2f}, neg_neg_centroid_cosine: {:.2f}, TL_N: {:.2f}, recall@25: {:.2f}").format(epoch+1,
                                                                                                         h[0],
                                                                                                         h[1], h[2], h[9], 
                                                                                                         h[3], h[4], h[10],
                                                                                                         h[5], h[6], h[11],
                                                                                                         h[7], h[8], h[12],
                                                                                                         recall))
    else:
        print(("Epoch: {}\n" + 
              "Loss: {:.2f}, pos_cosine: {:.2f}, neg_cosine: {:.2f}, TL: {:.2f}\n" +
              "anchor_pos_centroid_cosine: {:.2f}, anchor_neg_centroid_cosine: {:.2f}, TL_A: {:.2f}\n" +
              "pos_pos_centroid_cosine: {:.2f}, pos_neg_centroid_cosine: {:.2f}, TL_P: {:.2f}\n" +
              "neg_pos_centroid_cosine: {:.2f}, neg_neg_centroid_cosine: {:.2f}, TL_N: {:.2f}").format(epoch+1,
                                                                                                         h[0],
                                                                                                         h[1], h[2], h[9], 
                                                                                                         h[3], h[4], h[10],
                                                                                                         h[5], h[6], h[11],
                                                                                                         h[7], h[8], h[12]))
    loss = h[0]
    
    if loss < best_loss:
        best_loss = loss
        best_epoch = epoch+1

experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
experiment.save_model(encoded_anchor, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)
print('Best_epoch={}, Best_loss={:.2f}, Recall@25={:.2f}'.format(best_epoch, best_loss, recall))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 1682)         0                                            
__________________________________________________________________________________________________
title_in (InputLayer)           (None, 20)           0                                            
__________________________________________________________________________________________________
desc_in (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
info_pos (InputLayer)           (None, 1682)         0                                            
__________________________________________________________________________________________________
title_pos 

Epoch: 1
Loss: 1.02, pos_cosine: 0.92, neg_cosine: 0.94, TL: 0.98
anchor_pos_centroid_cosine: 0.97, anchor_neg_centroid_cosine: 0.94, TL_A: 1.03
pos_pos_centroid_cosine: 0.97, pos_neg_centroid_cosine: 0.92, TL_P: 1.05
neg_pos_centroid_cosine: 0.97, neg_neg_centroid_cosine: 0.94, TL_N: 1.03
Epoch: 2
Loss: 1.02, pos_cosine: 0.92, neg_cosine: 0.95, TL: 0.98
anchor_pos_centroid_cosine: 0.97, anchor_neg_centroid_cosine: 0.94, TL_A: 1.03
pos_pos_centroid_cosine: 0.97, pos_neg_centroid_cosine: 0.92, TL_P: 1.05
neg_pos_centroid_cosine: 0.97, neg_neg_centroid_cosine: 0.95, TL_N: 1.03
Epoch: 3
Loss: 1.02, pos_cosine: 0.94, neg_cosine: 0.96, TL: 0.99
anchor_pos_centroid_cosine: 0.98, anchor_neg_centroid_cosine: 0.95, TL_A: 1.02
pos_pos_centroid_cosine: 0.98, pos_neg_centroid_cosine: 0.94, TL_P: 1.04
neg_pos_centroid_cosine: 0.98, neg_neg_centroid_cosine: 0.96, TL_N: 1.02
Epoch: 4
Loss: 1.01, pos_cosine: 0.95, neg_cosine: 0.97, TL: 0.99
anchor_pos_centroid_cosine: 0.98, anchor_neg_centroid_cosine:

Epoch: 30
Loss: 0.99, pos_cosine: 0.95, neg_cosine: 1.00, TL: 0.95
anchor_pos_centroid_cosine: 0.98, anchor_neg_centroid_cosine: 0.97, TL_A: 1.01
pos_pos_centroid_cosine: 0.98, pos_neg_centroid_cosine: 0.95, TL_P: 1.02
neg_pos_centroid_cosine: 0.97, neg_neg_centroid_cosine: 0.98, TL_N: 0.99
Epoch: 31
Loss: 0.99, pos_cosine: 0.96, neg_cosine: 1.00, TL: 0.96
anchor_pos_centroid_cosine: 0.98, anchor_neg_centroid_cosine: 0.98, TL_A: 1.01
pos_pos_centroid_cosine: 0.97, pos_neg_centroid_cosine: 0.96, TL_P: 1.01
neg_pos_centroid_cosine: 0.98, neg_neg_centroid_cosine: 0.98, TL_N: 1.00
Epoch: 32
Loss: 0.99, pos_cosine: 0.90, neg_cosine: 1.00, TL: 0.91
anchor_pos_centroid_cosine: 0.95, anchor_neg_centroid_cosine: 0.96, TL_A: 0.99
pos_pos_centroid_cosine: 0.98, pos_neg_centroid_cosine: 0.93, TL_P: 1.04
neg_pos_centroid_cosine: 0.96, neg_neg_centroid_cosine: 0.94, TL_N: 1.02
Epoch: 33
Loss: 0.98, pos_cosine: 0.88, neg_cosine: 1.00, TL: 0.88
anchor_pos_centroid_cosine: 0.96, anchor_neg_centroid_cos

Epoch: 59
Loss: 0.96, pos_cosine: 0.75, neg_cosine: 1.00, TL: 0.75
anchor_pos_centroid_cosine: 0.81, anchor_neg_centroid_cosine: 0.85, TL_A: 0.96
pos_pos_centroid_cosine: 0.76, pos_neg_centroid_cosine: 0.67, TL_P: 1.09
neg_pos_centroid_cosine: 0.85, neg_neg_centroid_cosine: 0.81, TL_N: 1.04
Epoch: 60
Loss: 0.95, pos_cosine: 0.61, neg_cosine: 1.00, TL: 0.61
anchor_pos_centroid_cosine: 0.78, anchor_neg_centroid_cosine: 0.88, TL_A: 0.90
pos_pos_centroid_cosine: 0.76, pos_neg_centroid_cosine: 0.56, TL_P: 1.20
neg_pos_centroid_cosine: 0.89, neg_neg_centroid_cosine: 0.78, TL_N: 1.11
Epoch: 61
Loss: 0.92, pos_cosine: 0.50, neg_cosine: 1.00, TL: 0.50
anchor_pos_centroid_cosine: 0.73, anchor_neg_centroid_cosine: 0.71, TL_A: 1.03
pos_pos_centroid_cosine: 0.65, pos_neg_centroid_cosine: 0.49, TL_P: 1.16
neg_pos_centroid_cosine: 0.71, neg_neg_centroid_cosine: 0.73, TL_N: 0.98
Epoch: 62
Loss: 0.96, pos_cosine: 0.57, neg_cosine: 1.00, TL: 0.57
anchor_pos_centroid_cosine: 0.68, anchor_neg_centroid_cos

Epoch: 88
Loss: 0.97, pos_cosine: 0.81, neg_cosine: 1.00, TL: 0.81
anchor_pos_centroid_cosine: 0.89, anchor_neg_centroid_cosine: 0.84, TL_A: 1.05
pos_pos_centroid_cosine: 0.87, pos_neg_centroid_cosine: 0.81, TL_P: 1.05
neg_pos_centroid_cosine: 0.84, neg_neg_centroid_cosine: 0.89, TL_N: 0.95
Epoch: 89
Loss: 0.95, pos_cosine: 0.75, neg_cosine: 1.00, TL: 0.75
anchor_pos_centroid_cosine: 0.87, anchor_neg_centroid_cosine: 0.81, TL_A: 1.06
pos_pos_centroid_cosine: 0.78, pos_neg_centroid_cosine: 0.75, TL_P: 1.03
neg_pos_centroid_cosine: 0.81, neg_neg_centroid_cosine: 0.87, TL_N: 0.94
Epoch: 90
Loss: 0.92, pos_cosine: 0.64, neg_cosine: 1.00, TL: 0.64
anchor_pos_centroid_cosine: 0.81, anchor_neg_centroid_cosine: 0.81, TL_A: 1.00
pos_pos_centroid_cosine: 0.80, pos_neg_centroid_cosine: 0.77, TL_P: 1.03
neg_pos_centroid_cosine: 0.81, neg_neg_centroid_cosine: 0.81, TL_N: 1.00
Epoch: 91
Loss: 0.97, pos_cosine: 0.83, neg_cosine: 1.00, TL: 0.83
anchor_pos_centroid_cosine: 0.89, anchor_neg_centroid_cos

...

Epoch: 873
Loss: 0.96, pos_cosine: 0.63, neg_cosine: 1.00, TL: 0.63
anchor_pos_centroid_cosine: 0.72, anchor_neg_centroid_cosine: 0.72, TL_A: 1.00
pos_pos_centroid_cosine: 0.78, pos_neg_centroid_cosine: 0.56, TL_P: 1.22
neg_pos_centroid_cosine: 0.72, neg_neg_centroid_cosine: 0.72, TL_N: 1.00
Epoch: 874
Loss: 0.95, pos_cosine: 0.61, neg_cosine: 1.00, TL: 0.61
anchor_pos_centroid_cosine: 0.75, anchor_neg_centroid_cosine: 0.79, TL_A: 0.97
pos_pos_centroid_cosine: 0.76, pos_neg_centroid_cosine: 0.59, TL_P: 1.18
neg_pos_centroid_cosine: 0.79, neg_neg_centroid_cosine: 0.75, TL_N: 1.03
Epoch: 875
Loss: 0.93, pos_cosine: 0.60, neg_cosine: 1.00, TL: 0.60
anchor_pos_centroid_cosine: 0.75, anchor_neg_centroid_cosine: 0.63, TL_A: 1.12
pos_pos_centroid_cosine: 0.76, pos_neg_centroid_cosine: 0.65, TL_P: 1.11
neg_pos_centroid_cosine: 0.63, neg_neg_centroid_cosine: 0.75, TL_N: 0.88
Epoch: 876
Loss: 0.91, pos_cosine: 0.54, neg_cosine: 1.00, TL: 0.54
anchor_pos_centroid_cosine: 0.79, anchor_neg_centroid

Epoch: 901
Loss: 0.95, pos_cosine: 0.64, neg_cosine: 1.00, TL: 0.64
anchor_pos_centroid_cosine: 0.71, anchor_neg_centroid_cosine: 0.80, TL_A: 0.91
pos_pos_centroid_cosine: 0.77, pos_neg_centroid_cosine: 0.62, TL_P: 1.15
neg_pos_centroid_cosine: 0.80, neg_neg_centroid_cosine: 0.70, TL_N: 1.09
Epoch: 902
Loss: 0.94, pos_cosine: 0.64, neg_cosine: 1.00, TL: 0.64
anchor_pos_centroid_cosine: 0.74, anchor_neg_centroid_cosine: 0.62, TL_A: 1.12
pos_pos_centroid_cosine: 0.76, pos_neg_centroid_cosine: 0.64, TL_P: 1.11
neg_pos_centroid_cosine: 0.63, neg_neg_centroid_cosine: 0.75, TL_N: 0.88
Epoch: 903
Loss: 0.96, pos_cosine: 0.63, neg_cosine: 1.00, TL: 0.64
anchor_pos_centroid_cosine: 0.78, anchor_neg_centroid_cosine: 0.76, TL_A: 1.02
pos_pos_centroid_cosine: 0.79, pos_neg_centroid_cosine: 0.61, TL_P: 1.18
neg_pos_centroid_cosine: 0.77, neg_neg_centroid_cosine: 0.77, TL_N: 0.99
Epoch: 904
Loss: 0.95, pos_cosine: 0.52, neg_cosine: 1.00, TL: 0.52
anchor_pos_centroid_cosine: 0.74, anchor_neg_centroid

Epoch: 929
Loss: 0.96, pos_cosine: 0.63, neg_cosine: 1.00, TL: 0.63
anchor_pos_centroid_cosine: 0.74, anchor_neg_centroid_cosine: 0.78, TL_A: 0.95
pos_pos_centroid_cosine: 0.77, pos_neg_centroid_cosine: 0.57, TL_P: 1.20
neg_pos_centroid_cosine: 0.78, neg_neg_centroid_cosine: 0.74, TL_N: 1.05
Epoch: 930
Loss: 0.89, pos_cosine: 0.48, neg_cosine: 1.00, TL: 0.48
anchor_pos_centroid_cosine: 0.65, anchor_neg_centroid_cosine: 0.62, TL_A: 1.04
pos_pos_centroid_cosine: 0.70, pos_neg_centroid_cosine: 0.63, TL_P: 1.07
neg_pos_centroid_cosine: 0.62, neg_neg_centroid_cosine: 0.65, TL_N: 0.97
Epoch: 931
Loss: 0.96, pos_cosine: 0.59, neg_cosine: 1.00, TL: 0.59
anchor_pos_centroid_cosine: 0.71, anchor_neg_centroid_cosine: 0.70, TL_A: 1.00
pos_pos_centroid_cosine: 0.77, pos_neg_centroid_cosine: 0.53, TL_P: 1.24
neg_pos_centroid_cosine: 0.70, neg_neg_centroid_cosine: 0.71, TL_N: 1.00
Epoch: 932
Loss: 0.94, pos_cosine: 0.56, neg_cosine: 1.00, TL: 0.56
anchor_pos_centroid_cosine: 0.77, anchor_neg_centroid

Epoch: 957
Loss: 0.92, pos_cosine: 0.49, neg_cosine: 1.00, TL: 0.49
anchor_pos_centroid_cosine: 0.66, anchor_neg_centroid_cosine: 0.74, TL_A: 0.92
pos_pos_centroid_cosine: 0.77, pos_neg_centroid_cosine: 0.59, TL_P: 1.18
neg_pos_centroid_cosine: 0.74, neg_neg_centroid_cosine: 0.65, TL_N: 1.08
Epoch: 958
Loss: 0.94, pos_cosine: 0.59, neg_cosine: 0.99, TL: 0.60
anchor_pos_centroid_cosine: 0.70, anchor_neg_centroid_cosine: 0.66, TL_A: 1.04
pos_pos_centroid_cosine: 0.67, pos_neg_centroid_cosine: 0.52, TL_P: 1.15
neg_pos_centroid_cosine: 0.66, neg_neg_centroid_cosine: 0.70, TL_N: 0.96
Epoch: 959
Loss: 0.93, pos_cosine: 0.56, neg_cosine: 1.00, TL: 0.56
anchor_pos_centroid_cosine: 0.79, anchor_neg_centroid_cosine: 0.87, TL_A: 0.92
pos_pos_centroid_cosine: 0.78, pos_neg_centroid_cosine: 0.62, TL_P: 1.16
neg_pos_centroid_cosine: 0.87, neg_neg_centroid_cosine: 0.78, TL_N: 1.09
Epoch: 960
Loss: 0.92, pos_cosine: 0.59, neg_cosine: 1.00, TL: 0.59
anchor_pos_centroid_cosine: 0.65, anchor_neg_centroid

Epoch: 985
Loss: 0.98, pos_cosine: 0.70, neg_cosine: 1.00, TL: 0.70
anchor_pos_centroid_cosine: 0.80, anchor_neg_centroid_cosine: 0.80, TL_A: 1.00
pos_pos_centroid_cosine: 0.87, pos_neg_centroid_cosine: 0.66, TL_P: 1.22
neg_pos_centroid_cosine: 0.80, neg_neg_centroid_cosine: 0.80, TL_N: 1.00
Epoch: 986
Loss: 0.94, pos_cosine: 0.63, neg_cosine: 1.00, TL: 0.63
anchor_pos_centroid_cosine: 0.77, anchor_neg_centroid_cosine: 0.75, TL_A: 1.02
pos_pos_centroid_cosine: 0.76, pos_neg_centroid_cosine: 0.61, TL_P: 1.15
neg_pos_centroid_cosine: 0.75, neg_neg_centroid_cosine: 0.77, TL_N: 0.98
Epoch: 987
Loss: 0.95, pos_cosine: 0.69, neg_cosine: 1.00, TL: 0.70
anchor_pos_centroid_cosine: 0.81, anchor_neg_centroid_cosine: 0.82, TL_A: 1.00
pos_pos_centroid_cosine: 0.82, pos_neg_centroid_cosine: 0.72, TL_P: 1.10
neg_pos_centroid_cosine: 0.81, neg_neg_centroid_cosine: 0.81, TL_N: 1.00
Epoch: 988
Loss: 0.95, pos_cosine: 0.66, neg_cosine: 1.00, TL: 0.66
anchor_pos_centroid_cosine: 0.73, anchor_neg_centroid

In [38]:
recall

0.18

In [39]:
_[:20]

['327681:324658|378274:0.9603236466646194,388300:0.9603204466402531,326495:0.9602243788540363,337422:0.960174236446619,342957:0.9601263888180256,328404:0.9600710198283195,389961:0.959960300475359,322138:0.959914393723011,324648:0.9598733559250832,404846:0.9598314054310322,365821:0.9597340822219849,346680:0.9597157426178455,362157:0.9596523866057396,360060:0.9596004746854305,386745:0.9595713391900063,380617:0.9595657661557198,378446:0.9595654308795929,386766:0.9595603086054325,361152:0.9595353417098522,329381:0.9595335200428963,345610:0.959533017128706,315086:0.9595150947570801,391117:0.9595126584172249,316213:0.9595086611807346,361154:0.9594959579408169,308579:0.9594863429665565,380478:0.9594694934785366,378973:0.9594504050910473,343939:0.9594370201230049',
 '360457:362252|353496:0.9847983960062265,367431:0.9622252807021141,343568:0.9504827223718166,343310:0.9491984993219376,378207:0.9491901621222496,408751:0.9483517855405807,406736:0.9482144862413406,422502:0.948134295642376,418000:0.

In [40]:
# '''
#     Between 0-10 epochs recall@25 = 0.28
#     Between 0-20 epochs recall@25 = 0.32
#     Between 0-70 epochs recall@25 = ?
#     Between 0-100 epochs recall@25 = ?
# '''
# recall, exported_rank = experiment.evaluate_validation_test(retrieval, verbose, encoded_anchor, issues_by_buckets, bug_train_ids)

# "recall@25 last epoch:", recall

### Retrieval evaluation

In [41]:
print("Total of queries:", len(retrieval.test))

Total of queries: 4641


#### Getting the model trained

In [42]:
SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs))

'DMS_QL_1000_feature_1000epochs_64batch(eclipse)'

In [43]:
model = encoded_anchor
# model = experiment.get_model_vectorizer(path=SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)))

In [44]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 1682)         0                                            
__________________________________________________________________________________________________
title_in (InputLayer)           (None, 20)           0                                            
__________________________________________________________________________________________________
desc_in (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
FeatureMlpGenerationModel (Mode (None, 300)          504900      info_in[0][0]                    
__________________________________________________________________________________________________
FeatureLst

In [45]:
recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, 0, model, issues_by_buckets, bug_train_ids)

In [46]:
len(exported_rank)

7917

In [47]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))
EXPORT_RANK_PATH

'data/processed/eclipse/exported_rank_DMS_QL_1000.txt'

In [48]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in _:
        file_out.write(row + "\n")

In [49]:
report = experiment.evaluation.evaluate(EXPORT_RANK_PATH)
report

{'1 - recall_at_5': 0.13,
 '2 - recall_at_10': 0.15,
 '3 - recall_at_15': 0.16,
 '4 - recall_at_20': 0.17,
 '5 - recall_at_25': 0.18}

#### Some ideas to visualizate

- https://towardsdatascience.com/building-a-recommendation-system-using-neural-network-embeddings-1ef92e5c80c9